In [1]:


from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score, f1_score, recall_score

def input_mlflow(cv_r_v,increment_number,X_train, y_train,pipeline,run_name,experiment_id,X_test,y_test):
    

    for wi in range(len(cv_r_v)):
        
        #====================skema===================
        cols_spec = []
        data_map = {
                'int64': 'integer',
                'float64': 'double',
                'bool': 'boolean',
                'str': 'string',
                'object': 'string',
                "date": 'datetime'
            }

        for name, dtype in X_train.dtypes.to_dict().items():
            cols_spec.append(ColSpec(name=name, type=data_map[str(dtype)]))
        input_schema = Schema(inputs=cols_spec)
        output_schema = Schema([ColSpec(name="label", type="string")])
        #parameter = ParamSpec(name="model_name", dtype="string", default="model1")
        #param_schema = ParamSchema(params=[parameter])
        model_signature = ModelSignature(inputs=input_schema, outputs=output_schema)#, params=param_schema)
        #print("MODEL SIGNATURE")
        #print(model_signature.to_dict())

        model_signature = infer_signature(X_train, y_train)#, params={"model_name": "model1"})
        #print("MODEL SIGNATURE")
        #print(model_signature.to_dict())

        
        
        
        
        #====================buat run baru===================
        run_name_with_increment = f"{run_name}__{increment_number}"
            # Membuka run MLflow
        with mlflow.start_run(run_name=run_name_with_increment, experiment_id=experiment_id) as run:
            # Mendapatkan run_id
            run_id = mlflow.active_run().info.run_id
        
        print(f"mlruns/{experiment_id}/{run_id}/artifacts/grid_search__{increment_number}")
        increment_number=increment_number+1
        
        
        #====================buat parameter model dan metric===================

        # Definisikan grid parameter untuk dicari
        param_grid = cv_r_v[wi]
        
        #====================model===================

        # Inisialisasi GridSearchCV
        grid_search = GridSearchCV(pipeline, param_grid, cv=5,  scoring='accuracy')

        # Lakukan pencarian grid
        grid_search.fit(X_train, y_train)
        
            
        #====================parameter===================
        pipe=grid_search.best_estimator_
        
        
        first_step_name = list(pipe.named_steps.keys())[0:len(pipe)]
        

        for i in range(len(pipe)):
            # Mendapatkan parameter dari langkah 'sca'
            nama=first_step_name[i]
            sca_params = pipe.get_params()[nama]

                # Membuka run MLflow
            with mlflow.start_run( experiment_id=experiment_id,run_id=run_id) as run:

                # Log parameter secara otomatis menggunakan loop
                for param_name, param_value in sca_params.get_params().items():
                    param_name=nama+'__'+param_name
                    #print(param_name,param_value)
                    mlflow.log_param(param_name, param_value)
                    
                    
                #====================metric===================
                #matric
                hasil_test=grid_search.predict(X_test)
 
                
                
                
                                # Menghitung akurasi
                accuracy = accuracy_score(y_test, hasil_test)

                # Menghitung F1 score
                f1 = f1_score(y_test, hasil_test,average='macro')

                # Menghitung recall
                recall = recall_score(y_test, hasil_test,average='macro')

                
                
                                # Menghitung precision
                precision = precision_score(y_test, hasil_test,average='macro')


   
                

                # log model 
                mlflow.sklearn.log_model(sk_model=grid_search, artifact_path="grid_search__"+str(increment_number-1),signature=model_signature)

                metrics = {
                    'mean_test_score':pd.DataFrame(grid_search.cv_results_)['mean_test_score'].values[0],
                    "accuracy": accuracy,
                    "f1": f1,
                    "recall": recall,
                    "precision": precision,
         
                }

                mlflow.log_metrics(metrics)
                
                
        mlflow.end_run()
    print('selesai')

        
    return increment_number


     

def ambil_best(grid_search,n):
    cv_r=grid_search.copy()
    cv_r_v=cv_r.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
    cv_r_v=cv_r_v.head(n)['params'].values

    for wi in range(len(cv_r_v)):

        for i in list(cv_r_v[wi]):

            cv_r_v[wi][i]=[cv_r_v[wi][i]]


    return cv_r_v


In [2]:
import mlflow.sklearn
from sklearn.metrics import classification_report, confusion_matrix


def test(model_paths,X_test2,y_test):
    ii=1
    for i, model_path in enumerate(model_paths, start=1):
        print(f'\n\n================================| model {ii} |==========================================')
        
        model = mlflow.sklearn.load_model(model_path)
        y_pred = model.predict(X_test2)
        print(f"Classification Report for model {model_path[-2:]}:")
        print(classification_report(y_test, y_pred))
        print(f"Confusion Matrix for model {model_path[-2:]}:\n", confusion_matrix(y_test, y_pred), '\n\n')
        
        ii=ii+1

## Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score, f1_score, recall_score
# import libraries
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats



import seaborn as sns
import statsmodels.api as sm
import pandas as pd



from scipy.stats import shapiro,normaltest,kstest,jarque_bera
import pingouin as pg
from pingouin import kruskal


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import mlflow



import mlflow

from mlflow.models.signature import ModelSignature
from mlflow.models.signature import infer_signature
from mlflow.types.schema import Schema
from mlflow.types.schema import ParamSchema
from mlflow.types.schema import ParamSpec
from mlflow.types.schema import ColSpec
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import pandas as pd
from typing import Tuple


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score


import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector as selector

In [4]:
from scipy.stats import chi2_contingency, fisher_exact

# prepare data 0

In [5]:
df = pd.read_csv("data/data_csv/data_bersih.csv")

In [6]:
df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
0,Adelie,Torgersen,39.10,18.7,181.0,3750.0
1,Adelie,Torgersen,39.50,17.4,186.0,3800.0
2,Adelie,Torgersen,40.30,18.0,195.0,3250.0
3,Adelie,Torgersen,44.45,17.3,197.0,4050.0
4,Adelie,Torgersen,36.70,19.3,193.0,3450.0


In [7]:
df.isna().sum()

species              0
island               0
culmen_length_mm     0
culmen_depth_mm      0
flipper_length_mm    0
body_mass_g          0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            343 non-null    object 
 1   island             343 non-null    object 
 2   culmen_length_mm   343 non-null    float64
 3   culmen_depth_mm    343 non-null    float64
 4   flipper_length_mm  343 non-null    float64
 5   body_mass_g        343 non-null    float64
dtypes: float64(4), object(2)
memory usage: 16.2+ KB


In [9]:
X=df.drop(columns=['species'])
y=df['species']

Xs=pd.get_dummies(X,drop_first=True)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(Xs,y, test_size=0.15, random_state=142)

In [11]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=101)
model.fit(X_train,y_train)
model.get_depth()

5

# prepare data

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=142)
X_train.head()

,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
185,Dream,51.0,18.8,203.0,4100.0
254,Biscoe,49.1,14.8,220.0,5150.0
62,Biscoe,37.6,17.0,185.0,3600.0
149,Dream,37.8,18.1,193.0,3750.0
54,Biscoe,34.5,18.1,187.0,2900.0


# make pipeline

In [13]:


numeric_features = selector(dtype_include='float64')
categorical_features = selector(dtype_include='object')


numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])


categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first', handle_unknown='error'))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('dt', DecisionTreeClassifier(random_state=101))])


# make param_grid 

In [ ]:
param_grid = {
    'dt__max_depth': list(range(2, 7)),
    'dt__criterion': ['gini', 'entropy'],
    'dt__min_samples_split': [2, 5, 10],
    'dt__min_samples_leaf': [1, 2, 4],
    'dt__max_features': [None, 'auto', 'sqrt', 'log2'],
    'dt__max_leaf_nodes': [None, 10, 20, 30],
    'dt__min_impurity_decrease': [0.0, 0.01, 0.1]
}

grid_search_dt = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search_dt.fit(X_train, y_train)


In [ ]:
df_1 = pd.DataFrame(grid_search_dt.cv_results_)
df_1.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_1 = df_1.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_1.head()

In [ ]:


data_ambil=ambil_best(df_1,10)


In [ ]:
data_ambil

In [ ]:
if __name__ == "__main__":

    experiment_id = mlflow.create_experiment(
        name="project_DecisionTreeClassifier_new",
        tags={"env": "dev", "version": "1.0.0"},
    )

    print(experiment_id)
    

mlflow.end_run()


In [ ]:
run_name = "log___"
increment_number = 1

In [20]:

increment_number=input_mlflow(data_ambil,increment_number,X_train, y_train,pipeline,run_name,experiment_id,X_test,y_test)

mlruns/776004161425800329/b90db1c4dafb4b65ab48c49cba21b830/artifacts/grid_search__1


C:\Users\ACER\anaconda3\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


mlruns/776004161425800329/24a59867967848528ca40fdb6991f463/artifacts/grid_search__2
mlruns/776004161425800329/953b3bcf771f4c57ac6bad253a39a2b6/artifacts/grid_search__3
mlruns/776004161425800329/f83180406a0946d5bb421f5776adfb2e/artifacts/grid_search__4
mlruns/776004161425800329/810d77ac339b4990b153414527a27816/artifacts/grid_search__5
mlruns/776004161425800329/eff4ea110e5340bc9c0aeafe0cb86d9a/artifacts/grid_search__6
mlruns/776004161425800329/bb255a61228a4da9b259f41d2c42580d/artifacts/grid_search__7
mlruns/776004161425800329/2ce4bcc555c940e4b0d1651ae92be233/artifacts/grid_search__8
mlruns/776004161425800329/d686423b105a4cd598c668d26f119cb6/artifacts/grid_search__9
mlruns/776004161425800329/8480e85685e045149d778fb76524d7f3/artifacts/grid_search__10
selesai


In [22]:
model_paths = [
  
    'mlruns/776004161425800329/b90db1c4dafb4b65ab48c49cba21b830/artifacts/grid_search__1',
    'mlruns/776004161425800329/24a59867967848528ca40fdb6991f463/artifacts/grid_search__2',
    'mlruns/776004161425800329/953b3bcf771f4c57ac6bad253a39a2b6/artifacts/grid_search__3',
    'mlruns/776004161425800329/f83180406a0946d5bb421f5776adfb2e/artifacts/grid_search__4',
    'mlruns/776004161425800329/810d77ac339b4990b153414527a27816/artifacts/grid_search__5',
    'mlruns/776004161425800329/eff4ea110e5340bc9c0aeafe0cb86d9a/artifacts/grid_search__6',
    'mlruns/776004161425800329/bb255a61228a4da9b259f41d2c42580d/artifacts/grid_search__7',
    'mlruns/776004161425800329/2ce4bcc555c940e4b0d1651ae92be233/artifacts/grid_search__8',
    'mlruns/776004161425800329/d686423b105a4cd598c668d26f119cb6/artifacts/grid_search__9',
    'mlruns/776004161425800329/8480e85685e045149d778fb76524d7f3/artifacts/grid_search__10'

    ]

test(model_paths,X_test,y_test)



================================| model 1 |==========================================
Classification Report for model _1:
              precision    recall  f1-score   support

      Adelie       0.97      0.97      0.97        32
   Chinstrap       0.93      1.00      0.96        13
      Gentoo       1.00      0.96      0.98        24

    accuracy                           0.97        69
   macro avg       0.97      0.98      0.97        69
weighted avg       0.97      0.97      0.97        69

Confusion Matrix for model _1:
 [[31  1  0]
 [ 0 13  0]
 [ 1  0 23]] 




================================| model 2 |==========================================
Classification Report for model _2:
              precision    recall  f1-score   support

      Adelie       0.89      0.97      0.93        32
   Chinstrap       0.92      0.85      0.88        13
      Gentoo       1.00      0.92      0.96        24

    accuracy                           0.93        69
   macro avg       0.93    

---